# Clasificación de imágenes

Este tutorial muestra cómo clasificar imágenes de flores utilizando un modelo `tf.keras.Sequential` y cargar datos utilizando `tf.keras.utils.image_dataset_from_directory`. Demuestra los siguientes conceptos:

- Carga eficaz de un conjunto de datos del disco.
- La identificación del sobreajuste y la aplicación de técnicas para mitigarlo, incluidos el aumento de datos y abandonarlos.

Este tutorial sigue un flujo de trabajo básico de aprendizaje automático:

1. Examinar y comprender los datos
2. Construirá una canalización de entrada
3. Construir el modelo
4. Entrene el modelo
5. Pruebe el modelo
6. Mejore el modelo y repita el proceso

Además, el bloc de notas demuestra cómo convertir un [modelo guardado](../../../guide/saved_model.ipynb) en un [modelo de TensorFlow Lite](https://www.tensorflow.org/lite/) para el aprendizaje automático en dispositivos móviles, embebidos y de IoT.

## Preparación

Importe TensorFlow y otras bibliotecas necesarias:

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import PIL
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential

## Descargar y explorar el conjunto de datos

Este tutorial utiliza un conjunto de datos de unas 3,700 fotos de flores. El conjunto de datos contiene cinco subdirectorios, uno por clase:

```
flower_photo/
  daisy/
  dandelion/
  roses/
  sunflowers/
  tulips/
```

In [ ]:
import pathlib
import tensorflow as tf

dataset_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"
data_dir = tf.keras.utils.get_file('flower_photos', origin=dataset_url, extract=True)
data_dir = pathlib.Path(data_dir).parent / "flower_photos"  # Ajustamos la ruta extraída

# Contar imágenes correctamente
image_count = len(list(data_dir.glob('*/*/*.jpg')))
print(f"Total de imágenes: {image_count}")

Después de la descarga, ya debería disponer de una copia del conjunto de datos. Hay 3,670 imágenes en total:

Estas son algunas rosas:

In [ ]:
roses = list(data_dir.glob('flower_photos/roses/*'))
PIL.Image.open(str(roses[0]))

In [ ]:
PIL.Image.open(str(roses[1]))

Y algunos tulipanes:

In [ ]:
tulips = list(data_dir.glob('flower_photos/tulips/*'))
PIL.Image.open(str(tulips[0]))

In [ ]:
PIL.Image.open(str(tulips[1]))

## Cargar datos con una utilidad de Keras

A continuación, cargue estas imágenes desde el disco utilizando la útil utilidad `tf.keras.utils.image_dataset_from_directory`. Esto nos permitirá pasar de un directorio de imágenes en disco a un `tf.data.Dataset` con sólo un par de líneas de código. Si lo desea, también puede escribir su propio código para cargar datos desde cero visitando el tutorial [Cargar y preprocesar imágenes](../load_data/images.ipynb).

### Crear un conjunto de datos

Defina algunos parámetros para el cargador:

In [ ]:
batch_size = 32
img_height = 180
img_width = 180

Es una práctica recomendada utilizar una división de validación cuando desarrolle su modelo. Utilice el 80% de las imágenes para el entrenamiento y el 20% para la validación.

In [ ]:
train_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="training",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

In [ ]:
val_ds = tf.keras.utils.image_dataset_from_directory(
  data_dir,
  validation_split=0.2,
  subset="validation",
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Puede encontrar los nombres de las clases en el atributo `class_names` de estos conjuntos de datos. Éstos corresponden a los nombres de los directorios por orden alfabético.

In [ ]:
class_names = train_ds.class_names
print(class_names)

## Visualizar los datos

Estas son las nueve primeras imágenes del conjunto de datos de entrenamiento:

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for images, labels in train_ds.take(1):
  for i in range(9):
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(images[i].numpy().astype("uint8"))
    plt.title(class_names[labels[i]])
    plt.axis("off")

Pasará estos conjuntos de datos al método Keras `Model.fit` para el entrenamiento más adelante en este tutorial. Si lo desea, también puede iterar manualmente el conjunto de datos y recuperar lotes de imágenes:

In [ ]:
for image_batch, labels_batch in train_ds:
  print(image_batch.shape)
  print(labels_batch.shape)
  break

El lote de imagen `image_batch` es un tensor de la forma `(32, 180, 180, 3)`. Esto es un lote de 32 imágenes de forma `180x180x3` (la última dimensión hace referencia a los canales de color RGB). El lote `label_batch` es un tensor de la forma `(32,)`, estas son etiquetas que concuerdan con las 32 imágenes.

Puede llamar a `.numpy()` sobre los tensores `image_batch` y `labels_batch` para convertirlos en un `numpy.ndarray`.


## Configuración del conjunto de datos para rendimiento

Asegúrese de utilizar la preextracción de datos en búfer, para poder ceder los datos desde el disco sin que la E/S se bloquee. Estos son dos métodos importantes que debe utilizar al cargar datos:

- `Dataset.cache` conserva los datos en la memoria después de que se carga desde el disco durante la primera época. Así se garantiza que el conjunto de datos no se transforme en un cuello de botella mientras entrena su modelo. Si su conjunto de datos es muy grande para guardar en la memoria, también puede usar este método para crear un caché en disco de alto rendimiento.
- `Dataset.prefetch` superpone el preprocesamiento de los datos y la ejecución del modelo durante el entrenamiento.

Quienes quieran aprender más sobre ambos modelos y también sobre cómo copiar datos en caché en disco, pueden leer la sección *Preextracción* de la guía [Mejor rendimiento con la API tf.data](../../guide/data_performance.ipynb).

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE)
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)

## Estandarizar los datos

Los valores del canal RGB están dentro del rango `[0, 255]`, lo cual no es ideal para una red neuronal. En general, debería buscar que los valores de su entrada sean bajos.

Aquí, estandarizará los valores para que estén dentro del rango `[0, 1]` mediante el uso de `tf.keras.layers.Rescaling`:

In [ ]:
normalization_layer = layers.Rescaling(1./255)

Esta capa se puede usar de dos formas. Se puede aplicar en el conjunto de datos llamando `Dataset.map`:

In [ ]:
normalized_ds = train_ds.map(lambda x, y: (normalization_layer(x), y))
image_batch, labels_batch = next(iter(normalized_ds))
first_image = image_batch[0]
# Notice the pixel values are now in `[0,1]`.
print(np.min(first_image), np.max(first_image))

O bien, puede incluir la capa dentro de la definición de su modelo, lo que puede simplificar la implementación. Utilice aquí el segundo enfoque.

Nota: Previamente, usaste el argumento `image_size` de `tf.keras.utils.image_dataset_from_directory` para ajustar el tamaño de las imágenes. Si también quiere incluir la lógica del ajuste en su modelo, puede usar la capa `tf.keras.layers.Resizing`.

## Un modelo básico de Keras

### Crear el modelo

El modelo [Sequential](https://www.tensorflow.org/guide/keras/sequential_model) de Keras consta de tres bloques de convolución (`tf.keras.layers.Conv2D`) con una capa de agrupamiento máximo (`tf.keras.layers.MaxPooling2D`) en cada uno de ellos. Hay una capa totalmente conectada (`tf.keras.layers.Dense`) con 128 unidades en la parte superior que se activa mediante una función de activación ReLU (`'relu'`). Este modelo no se ajustó para obtener una gran precisión; el objetivo de este tutorial es mostrar un enfoque estándar.

In [ ]:
num_classes = len(class_names)

model = Sequential([
  layers.Rescaling(1./255, input_shape=(img_height, img_width, 3)),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

### Compilar el modelo

Para este tutorial, elija el optimizador `tf.keras.optimizers.Adam` y la función de pérdida `tf.keras.losses.SparseCategoricalCrossentropy`. Para ver la precisión de entrenamiento y validación de cada época de entrenamiento, pase el argumento `metrics` a `Model.compile`.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Resumen del modelo

Visualice todas las capas de la red utilizando el método Keras `Model.summary`:

In [ ]:
model.summary()

### Entrene el modelo

Entrene el modelo durante 10 épocas con el método Keras `Model.fit`:

In [ ]:
epochs=1
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

## Visualice los resultados del entrenamiento

Cree gráficos de la pérdida y la precisión en los conjuntos de entrenamiento y validación:

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

Las representaciones gráficas muestran que la precisión de entrenamiento y la de validación están alejadas por grandes márgenes, y que el modelo sólo ha logrado una precisión en torno al 60% en el conjunto de validación.

Las siguientes secciones del tutorial muestran cómo inspeccionar lo que salió mal e intentar aumentar el rendimiento general del modelo.

## Sobreajuste

En las representaciones gráficas anteriores, la precisión de entrenamiento aumenta linealmente con el tiempo, mientras que la precisión de validación se estanca en torno al 60% en el proceso de entrenamiento. Además, la diferencia de precisión entre la precisión de entrenamiento y la de validación es notable, un signo de [sobreajuste](https://www.tensorflow.org/tutorials/keras/overfit_and_underfit).

Cuando hay un número reducido de ejemplos de entrenamiento, el modelo aprende a veces de los ruidos o de los detalles no deseados de los ejemplos de entrenamiento, hasta el punto de repercutir negativamente en el rendimiento del modelo con los nuevos ejemplos. Este fenómeno se conoce como sobreajuste. Significa que el modelo tendrá dificultades para generalizar en un nuevo conjunto de datos.

Hay múltiples formas de combatir el sobreajuste en el proceso de entrenamiento. En este tutorial, utilizará el *aumento de datos* y agregará *abandonar* a su modelo.

## Aumento de datos

El sobreajuste suele producirse cuando hay un número reducido de ejemplos en el entrenamiento. [El aumento de datos](./data_augmentation.ipynb) adopta el enfoque consistente en generar datos de entrenamiento adicionales a partir de los ejemplos existentes, al aumentarlos mediante transformaciones aleatorias que producen imágenes de aspecto realista. Esto ayuda a exponer el modelo a más aspectos de los datos y a generalizar mejor.

Implementará el aumento de datos utilizando las siguientes capas de preprocesamiento de Keras: `tf.keras.layers.RandomFlip`, `tf.keras.layers.RandomRotation`, y `tf.keras.layers.RandomZoom`. Estas pueden incluirse dentro de su modelo como otras capas, y ejecutarse en la GPU.

In [ ]:
data_augmentation = keras.Sequential(
  [
    layers.RandomFlip("horizontal",
                      input_shape=(img_height,
                                  img_width,
                                  3)),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1),
  ]
)

Visualice algunos ejemplos aumentados aplicando varias veces el aumento de datos a la misma imagen:

In [ ]:
plt.figure(figsize=(10, 10))
for images, _ in train_ds.take(1):
  for i in range(9):
    augmented_images = data_augmentation(images)
    ax = plt.subplot(3, 3, i + 1)
    plt.imshow(augmented_images[0].numpy().astype("uint8"))
    plt.axis("off")

En el siguiente paso agregará el aumento de datos a su modelo antes del entrenamiento.

## Abandonar

Otra técnica para reducir el sobreajuste es introducir [abandonar](https://developers.google.com/machine-learning/glossary#dropout_regularization){:.external} regularización a la red.

Cuando se aplica abandonar a una capa, se elimina aleatoriamente (poniendo la activación a cero) un número de unidades de salida de la capa durante el proceso de entrenamiento. Abandonar toma un número fraccionario como valor de entrada, en la forma como 0.1, 0.2, 0.4, etc. Esto significa descartar aleatoriamente el 10%, el 20% o el 40% de las unidades de salida de la capa aplicada.

Cree una nueva red neuronal con `tf.keras.layers.Dropout` antes de entrenarla utilizando las imágenes aumentadas:

In [ ]:
model = Sequential([
  data_augmentation,
  layers.Rescaling(1./255),
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Dropout(0.2),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes, name="outputs")
])

## Compile y entrene el modelo

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
epochs = 15
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

## Visualice los resultados del entrenamiento

Después de aplicar el aumento de datos y `tf.keras.layers.Dropout`, hay menos sobreajuste que antes, y la precisión de entrenamiento y validación están más alineadas:

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

## Predecir con nuevos datos

Utilice su modelo para clasificar una imagen que no estaba incluida en los conjuntos de entrenamiento o validación.

Nota: El aumento de datos y abandonar capas estarán inactivos en el momento de hacer inferencias.

In [ ]:
sunflower_url = "https://storage.googleapis.com/download.tensorflow.org/example_images/592px-Red_sunflower.jpg"
sunflower_path = tf.keras.utils.get_file('Red_sunflower', origin=sunflower_url)

img = tf.keras.utils.load_img(
    sunflower_path, target_size=(img_height, img_width)
)
img_array = tf.keras.utils.img_to_array(img)
img_array = tf.expand_dims(img_array, 0) # Create a batch

predictions = model.predict(img_array)
score = tf.nn.softmax(predictions[0])

print(
    "This image most likely belongs to {} with a {:.2f} percent confidence."
    .format(class_names[np.argmax(score)], 100 * np.max(score))
)

## Utilice TensorFlow Lite

TensorFlow Lite es un conjunto de herramientas que permite el aprendizaje automático en el dispositivo ayudando a los desarrolladores a ejecutar sus modelos en dispositivos móviles, integrados y edge.

### Convertir el modelo secuencial Keras en un modelo de TensorFlow Lite

Para utilizar el modelo entrenado con aplicaciones en el dispositivo, primero [convertirlo](https://www.tensorflow.org/lite/models/convert) a un formato de modelo más pequeño y eficiente llamado modelo [TensorFlow Lite](https://www.tensorflow.org/lite/).

En este ejemplo, tome el modelo secuencial de Keras entrenado y utilice `tf.lite.TFLiteConverter.from_keras_model` para generar un modelo [TensorFlow Lite](https://www.tensorflow.org/lite/):

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)

El modelo TensorFlow Lite que guardó en el paso anterior puede contener varias firmas de funciones. La API de conversión de modelos de Keras utiliza automáticamente la firma predeterminada. Obtenga más información sobre [firmas de TensorFlow Lite](https://www.tensorflow.org/lite/guide/signatures).

# Aumentación de datos

## Descripción general

Este tutorial muestra la aumentación de datos: una técnica para aumentar la diversidad de su conjunto de entrenamiento aplicando transformaciones aleatorias (pero realistas), como la rotación de imágenes.

Aprenderá a aplicar la aumentación de datos de dos maneras:

- Usar las capas de preprocesamiento Keras, como `tf.keras.layers.Resizing`, `tf.keras.layers.Rescaling`, `tf.keras.layers.RandomFlip`, y `tf.keras.layers.RandomRotation`.
- Usar los métodos `tf.image`, como `tf.image.flip_left_right`, `tf.image.rgb_to_grayscale`, `tf.image.adjust_brightness`, `tf.image.central_crop`, y `tf.image.stateless_random*`.

## Preparación

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras import layers

## Descargar un conjunto de datos

Este tutorial usa el conjunto de datos [tf_flowers](https://www.tensorflow.org/datasets/catalog/tf_flowers). Para mayor comodidad, descargue el conjunto de datos utilizando [Conjuntos de datos de TensorFlow](https://www.tensorflow.org/datasets). Si desea conocer otras formas de importar datos, consulte el tutorial [cargar imágenes](https://www.tensorflow.org/tutorials/load_data/images).


In [ ]:
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

El conjunto de datos de flores tiene cinco clases.

In [ ]:
num_classes = metadata.features['label'].num_classes
print(num_classes)

Recuperemos una imagen del conjunto de datos y usémosla para demostrar la aumentación de datos.

In [ ]:
get_label_name = metadata.features['label'].int2str

image, label = next(iter(train_ds))
_ = plt.imshow(image)
_ = plt.title(get_label_name(label))

## Usar las capas de preprocesamiento Keras

### Redimensionar y reescalar


Puede usar las capas de preprocesamiento de Keras para redimensionar sus imágenes de forma consistente (con `tf.keras.layers.Resizing`), y para reescalar los valores de los pixeles (con `tf.keras.layers.Rescaling`).

In [ ]:
IMG_SIZE = 180

resize_and_rescale = tf.keras.Sequential([
  layers.Resizing(IMG_SIZE, IMG_SIZE),
  layers.Rescaling(1./255)
])

Nota: La capa de reescalado anterior estandariza los valores de los pixeles al rango `[0, 1]`. Si en su lugar quisiera que fuera `[-1, 1]`, pudiese escribir `tf.keras.layers.Rescaling(1./127.5, offset=-1)`.


Puede visualizar el resultado de aplicar estas capas a una imagen. 

In [ ]:
result = resize_and_rescale(image)
_ = plt.imshow(result)

Compruebe que los píxeles se encuentran en el intervalo `[0, 1]`:

In [ ]:
print("Min and max pixel values:", result.numpy().min(), result.numpy().max())

### Aumentación de datos

También puede usar las capas de preprocesamiento de Keras para aumentar datos, como `tf.keras.layers.RandomFlip` y `tf.keras.layers.RandomRotation`.

Creemos unas cuantas capas de preprocesamiento y apliquémoslas repetidamente a la misma imagen.

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.2),
])

In [ ]:
# Add the image to a batch.
image = tf.cast(tf.expand_dims(image, 0), tf.float32)

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
  augmented_image = data_augmentation(image)
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(augmented_image[0])
  plt.axis("off")

Puede usar diversas capas de preprocesamiento para aumentar datos, como `tf.keras.layers.RandomContrast`, `tf.keras.layers.RandomCrop`, `tf.keras.layers.RandomZoom`, y otras.

### Dos opciones para usar las capas de preprocesamiento de Keras

Hay dos formas de usar estas capas de preprocesamiento, con importantes concesiones.

#### Aplicar las capas de preprocesamiento a su conjunto de datos

In [ ]:
aug_ds = train_ds.map(
  lambda x, y: (resize_and_rescale(x, training=True), y))

Con este enfoque, se usa `Dataset.map` para crear un conjunto de datos que produzca lotes de imágenes aumentadas. En este caso:

- La aumentación de datos se producirá de forma asíncrona en la CPU y no se bloqueará. Puede superponer el entrenamiento de su modelo en la GPU con el preprocesamiento de datos, usando `Dataset.prefetch`, que se muestra a continuación.
- En este caso, las capas de preprocesamiento no se exportarán con el modelo cuando llame a `Model.save`. Deberá adjuntarlas a su modelo antes de guardarlo o reimplementarlas en el lado del servidor. Después del entrenamiento, puede adjuntar las capas de preprocesamiento antes de exportarlas.


Puede encontrar un ejemplo de la primera opción en el tutorial [Clasificación de imágenes](classification.ipynb). Vamos a demostrar aquí la segunda opción.

### Aplicar las capas de preprocesamiento a los conjuntos de datos

Configure los conjuntos de datos de entrenamiento, validación y prueba con las capas de preprocesamiento Keras que creó anteriormente. También configurará los conjuntos de datos para mejorar el rendimiento, usando lecturas paralelas y preextracción en búfer para obtener lotes del disco sin que la E/S se bloquee. (Obtenga más información sobre el rendimiento de los conjuntos de datos en la guía [Mejor rendimiento con la API tf.data](https://www.tensorflow.org/guide/data_performance)).

Nota: La aumentación de datos sólo debe aplicarse al conjunto de entrenamiento.

In [ ]:
batch_size = 32
AUTOTUNE = tf.data.AUTOTUNE

def prepare(ds, shuffle=False, augment=False):
  # Resize and rescale all datasets.
  ds = ds.map(lambda x, y: (resize_and_rescale(x), y), 
              num_parallel_calls=AUTOTUNE)

  if shuffle:
    ds = ds.shuffle(1000)

  # Batch all datasets.
  ds = ds.batch(batch_size)

  # Use data augmentation only on the training set.
  if augment:
    ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=AUTOTUNE)

  # Use buffered prefetching on all datasets.
  return ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
train_ds = prepare(train_ds, shuffle=True, augment=True)
val_ds = prepare(val_ds)
test_ds = prepare(test_ds)

### Entrenar un modelo

Para completar, ahora entrenará un modelo usando los conjuntos de datos que acaba de preparar.

El modelo [Secuencial](https://www.tensorflow.org/guide/keras/sequential_model) consta de tres bloques de convolución (`tf.keras.layers.Conv2D`) con una capa de agrupamiento máximo (`tf.keras.layers.MaxPooling2D`) en cada uno de ellos. Hay una capa (`tf.keras.layers.Dense`) totalmente conectada (`tf.keras.layers.Dense`) con 128 unidades que se activa mediante una función de activación ReLU (`'relu'`). Este modelo no se ha ajustado para obtener precisión (la meta es mostrarle la mecánica).

In [ ]:
model = tf.keras.Sequential([
  layers.Conv2D(16, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(32, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, padding='same', activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

Seleccione el optimizador `tf.keras.optimizers.Adam` y la función de pérdida `tf.keras.losses.SparseCategoricalCrossentropy`. Para ver la precisión del entrenamiento y la validación de cada época de entrenamiento, pase el argumento `metrics` a `Model.compile`.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

Entrene por algunas épocas:

In [ ]:
epochs=5
history = model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=epochs
)

In [ ]:
loss, acc = model.evaluate(test_ds)
print("Accuracy", acc)

### Aumentación de datos personalizados

También puede crear capas de aumentación de datos personalizadas.

Esta sección del tutorial muestra dos formas de hacerlo:

- En primer lugar, creará una capa `tf.keras.layers.Lambda`. Esta es una buena manera de escribir código conciso.
- Luego, escribirá una nueva capa por [subclaseado](https://www.tensorflow.org/guide/keras/custom_layers_and_models), lo que le da más control.

Ambas capas invertirán aleatoriamente los colores de una imagen, según cierta probabilidad.

In [ ]:
def random_invert_img(x, p=0.5):
  if  tf.random.uniform([]) < p:
    x = (255-x)
  else:
    x
  return x

In [ ]:
def random_invert(factor=0.5):
  return layers.Lambda(lambda x: random_invert_img(x, factor))

random_invert = random_invert()

In [ ]:
plt.figure(figsize=(10, 10))
for i in range(9):
  augmented_image = random_invert(image)
  ax = plt.subplot(3, 3, i + 1)
  plt.imshow(augmented_image[0].numpy().astype("uint8"))
  plt.axis("off")

En seguida, implemente una capa personalizada a través de [subclaseado](https://www.tensorflow.org/guide/keras/custom_layers_and_models):

In [ ]:
class RandomInvert(layers.Layer):
  def __init__(self, factor=0.5, **kwargs):
    super().__init__(**kwargs)
    self.factor = factor

  def call(self, x):
    return random_invert_img(x)

In [ ]:
_ = plt.imshow(RandomInvert()(image)[0])

Ambas capas pueden usarse como se describe en las opciones 1 y 2 anteriores.

## Usar tf.image

Las utilidades de preprocesamiento Keras anteriores son convenientes. Pero, para un control más fino, puede escribir sus propias canalizaciones o capas de aumentación de datos usando `tf.data` y `tf.image` (también puede consultar [Complemento de TensorFlow Imagen: Operaciones](https://www.tensorflow.org/addons/tutorials/image_ops) y [TensorFlow I/O: Conversiones de Espacio de Color](https://www.tensorflow.org/io/tutorials/colorspace).)

Dado que el conjunto de datos de flores se configuró previamente con la aumentación de datos, vamos a reimportarlo para empezar de cero:

In [ ]:
(train_ds, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

Recupere una imagen con la que trabajar:

In [ ]:
image, label = next(iter(train_ds))
_ = plt.imshow(image)
_ = plt.title(get_label_name(label))

Usemos la siguiente función para visualizar y comparar las imágenes original y aumentada una al lado de la otra:

In [ ]:
def visualize(original, augmented):
  fig = plt.figure()
  plt.subplot(1,2,1)
  plt.title('Original image')
  plt.imshow(original)

  plt.subplot(1,2,2)
  plt.title('Augmented image')
  plt.imshow(augmented)

### Aumentación de datos

#### Invertir una imagen

Invierta una imagen vertical u horizontalmente con `tf.image.flip_left_right`:

In [ ]:
flipped = tf.image.flip_left_right(image)
visualize(image, flipped)

#### Aplicar escala de grises a una imagen

Puede aplicar escala de grises a una imagen con `tf.image.rgb_to_grayscale`:

In [ ]:
grayscaled = tf.image.rgb_to_grayscale(image)
visualize(image, tf.squeeze(grayscaled))
_ = plt.colorbar()

#### Saturar una imagen

Sature una imagen con `tf.image.adjust_saturation` indicando un factor de saturación:

In [ ]:
saturated = tf.image.adjust_saturation(image, 3)
visualize(image, saturated)

#### Cambiar el brillo de la imagen

Cambie el brillo de la imagen con `tf.image.adjust_brightness` indicando un factor de brillo:

In [ ]:
bright = tf.image.adjust_brightness(image, 0.4)
visualize(image, bright)

#### Recortar una imagen al centro

Recorte la imagen desde el centro hasta la parte de la imagen que desee usando `tf.image.central_crop`:

In [ ]:
cropped = tf.image.central_crop(image, central_fraction=0.5)
visualize(image, cropped)

#### Girar una imagen

Gire una imagen 90 grados con `tf.image.rot90`:

In [ ]:
rotated = tf.image.rot90(image)
visualize(image, rotated)

### Transformaciones aleatorias

Advertencia: Existen dos conjuntos de operaciones de imagen aleatoria: `tf.image.random*` y `tf.image.stateless_random*`. Se desaconseja encarecidamente usar las operaciones `tf.image.random*`, ya que usan los antiguos RNG de TF 1.x. En su lugar, use las operaciones de imagen aleatoria introducidas en este tutorial. Para más información, consulte [Generación de números aleatorios](../../guide/random_numbers.ipynb).

Aplicar transformaciones aleatorias a las imágenes puede ayudar aún más a generalizar y ampliar el conjunto de datos. La API actual `tf.image` ofrece ocho operaciones (ops) de imagen aleatorias de este tipo:

- [`tf.image.stateless_random_brightness`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_brightness)
- [`tf.image.stateless_random_contrast`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_contrast)
- [`tf.image.stateless_random_crop`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_crop)
- [`tf.image.stateless_random_flip_left_right`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_flip_left_right)
- [`tf.image.stateless_random_flip_up_down`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_flip_up_down)
- [`tf.image.stateless_random_hue`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_hue)
- [`tf.image.stateless_random_jpeg_quality`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_jpeg_quality)
- [`tf.image.stateless_random_saturation`](https://www.tensorflow.org/api_docs/python/tf/image/stateless_random_saturation)

Estas ops de imagen aleatoria son puramente funcionales: la salida sólo depende de la entrada. Esto hace que sean fáciles de usar en canalizaciones de entrada deterministas de alto rendimiento. Requieren que se introduzca un valor `seed` en cada paso. Dada la misma `seed`, devuelven los mismos resultados independientemente de cuántas veces se les llame.

Nota: `seed` es un `Tensor` de forma `(2,)` cuyos valores son cualesquiera enteros.

En las siguientes secciones, usted:

1. Repasará ejemplos de cómo usar operaciones de imagen aleatorias para transformar una imagen.
2. Demuestre cómo aplicar transformaciones aleatorias a un conjunto de datos de entrenamiento.

#### Modificar aleatoriamente el brillo de la imagen

Modifique aleatoriamente el brillo de `image` usando `tf.image.stateless_random_brightness` al dar un factor de brillo y `seed`. El factor de brillo se elige aleatoriamente en el rango `[-max_delta, max_delta)` y se asocia a la `seed` dada.

In [ ]:
for i in range(3):
  seed = (i, 0)  # tuple of size (2,)
  stateless_random_brightness = tf.image.stateless_random_brightness(
      image, max_delta=0.95, seed=seed)
  visualize(image, stateless_random_brightness)

#### Cambiar aleatoriamente el contraste de la imagen

Cambie aleatoriamente el contraste de `image` usando `tf.image.stateless_random_contrast` al dar un intervalo de contraste y `seed`. El rango de contraste se elige aleatoriamente en el intervalo `[lower, upper]` y se asocia con la `seed` dada.

In [ ]:
for i in range(3):
  seed = (i, 0)  # tuple of size (2,)
  stateless_random_contrast = tf.image.stateless_random_contrast(
      image, lower=0.1, upper=0.9, seed=seed)
  visualize(image, stateless_random_contrast)

#### Recortar una imagen al azar

Recorte aleatoriamente `imagen` usando `tf.image.stateless_random_crop` al dar un `size` y `seed` del objetivo. La parte que se recorta de `image` está en un punto elegido aleatoriamente y se asocia a la `seed` dada.

In [ ]:
for i in range(3):
  seed = (i, 0)  # tuple of size (2,)
  stateless_random_crop = tf.image.stateless_random_crop(
      image, size=[210, 300, 3], seed=seed)
  visualize(image, stateless_random_crop)

### Aplicar la aumentación a un conjunto de datos

En primer lugar, descarguemos de nuevo el conjunto de datos de imágenes por si se han modificado en las secciones anteriores.

In [ ]:
(train_datasets, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

A continuación, defina una función de utilidad para cambiar el tamaño y la escala de las imágenes. Esta función se usará para unificar el tamaño y la escala de las imágenes del conjunto de datos:

In [ ]:
def resize_and_rescale(image, label):
  image = tf.cast(image, tf.float32)
  image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
  image = (image / 255.0)
  return image, label

Definamos también la función `augment` que puede aplicar las transformaciones aleatorias a las imágenes. Esta función se usará sobre el conjunto de datos en el siguiente paso.

In [ ]:
def augment(image_label, seed):
  image, label = image_label
  image, label = resize_and_rescale(image, label)
  image = tf.image.resize_with_crop_or_pad(image, IMG_SIZE + 6, IMG_SIZE + 6)
  # Make a new seed.
  new_seed = tf.random.split(seed, num=1)[0, :]
  # Random crop back to the original size.
  image = tf.image.stateless_random_crop(
      image, size=[IMG_SIZE, IMG_SIZE, 3], seed=seed)
  # Random brightness.
  image = tf.image.stateless_random_brightness(
      image, max_delta=0.5, seed=new_seed)
  image = tf.clip_by_value(image, 0, 1)
  return image, label

#### Opción 1: Usar tf.data.experimental.Counter

Cree un objeto `tf.data.experimental.Counter` (llamémoslo `counter`) y `Dataset.zip` se llamará el conjunto de datos con `(counter, counter)`. Esto asegurará que cada imagen del conjunto de datos se asocie con un valor único (de forma `(2,)`) basado en `counter` que más tarde puede ser pasado a la función `augment` como el valor `seed` para transformaciones aleatorias.

In [ ]:
# Create a `Counter` object and `Dataset.zip` it together with the training set.
counter = tf.data.experimental.Counter()
train_ds = tf.data.Dataset.zip((train_datasets, (counter, counter)))

Mapee la función `augment` al conjunto de datos de entrenamiento:

In [ ]:
train_ds = (
    train_ds
    .shuffle(1000)
    .map(augment, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

In [ ]:
val_ds = (
    val_ds
    .map(resize_and_rescale, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

In [ ]:
test_ds = (
    test_ds
    .map(resize_and_rescale, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

#### Opción 2: Usar tf.random.Generator

- Cree un objeto `tf.random.Generator` con un valor inicial `seed`. Si se llama a la función `make_seeds` sobre el mismo objeto generador, siempre se devuelve un nuevo valor `seed` único.
- Defina una función contenedora que: 1) llame a la función `make_seeds`; y 2) pase el valor `seed` recién generado a la función `augment` para transformaciones aleatorias.

Nota: Los objetos `tf.random.Generator` almacenan el estado del RNG en una `tf.Variable`, lo que significa que puede guardarse como un [checkpoint](../../guide/checkpoint.ipynb) o en un [SavedModel](../../guide/saved_model.ipynb). Para más detalles, consulte [Generación de números aleatorios](../../guide/random_numbers.ipynb).

In [ ]:
# Create a generator.
rng = tf.random.Generator.from_seed(123, alg='philox')

In [ ]:
# Create a wrapper function for updating seeds.
def f(x, y):
  seed = rng.make_seeds(2)[0]
  image, label = augment((x, y), seed)
  return image, label

Mapee la función contenedora `f` al conjunto de datos de entrenamiento, y la función `resize_and_rescale` a los conjuntos de validación y prueba:

In [ ]:
train_ds = (
    train_datasets
    .shuffle(1000)
    .map(f, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

In [ ]:
val_ds = (
    val_ds
    .map(resize_and_rescale, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

In [ ]:
test_ds = (
    test_ds
    .map(resize_and_rescale, num_parallel_calls=AUTOTUNE)
    .batch(batch_size)
    .prefetch(AUTOTUNE)
)

Estos conjuntos de datos pueden usarse ahora para entrenar un modelo como se ha mostrado anteriormente.

## Siguientes pasos

Este tutorial demostró la aumentación de datos usando capas de preprocesamiento Keras y `tf.image`.

- Para aprender a incluir capas de preprocesamiento dentro de su modelo, consulte el tutorial [Clasificación de imágenes](classification.ipynb).
- También puede interesarle aprender cómo las capas de preprocesamiento pueden ayudarle a clasificar texto, como se muestra en el tutorial [Clasificación básica de texto](../keras/text_classification.ipynb).
- Puede aprender más sobre `tf.data` en esta [guía](../../guide/data.ipynb), y puede aprender a configurar sus canalizaciones de entrada para mejorar el rendimiento [aquí](../../guide/data_performance.ipynb).

##### Copyright 2019 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Dogs vs Cats Image Classification With Image Augmentation

In this tutorial, we will discuss how to classify images into pictures of cats or pictures of dogs. We'll build an image classifier using `tf.keras.Sequential` model and load data using `tf.keras.preprocessing.image.ImageDataGenerator`.

## Specific concepts that will be covered:
In the process, we will build practical experience and develop intuition around the following concepts

* Building _data input pipelines_ using the `tf.keras.preprocessing.image.ImageDataGenerator` class — How can we efficiently work with data on disk to interface with our model?
* _Overfitting_ - what is it, how to identify it, and how can we prevent it?
* _Data Augmentation_ and _Dropout_ - Key techniques to fight overfitting in computer vision tasks that we will incorporate into our data pipeline and image classifier model.

## We will follow the general machine learning workflow:

1. Examine and understand data
2. Build an input pipeline
3. Build our model
4. Train our model
5. Test our model
6. Improve our model/Repeat the process

<hr>

**Before you begin**

Before running the code in this notebook, reset the runtime by going to **Runtime -> Reset all runtimes** in the menu above. If you have been working through several notebooks, this will help you avoid reaching Colab's memory limits.


# Importing packages

Let's start by importing required packages:

*   os — to read files and directory structure
*   numpy — for some matrix math outside of TensorFlow
*   matplotlib.pyplot — to plot the graph and display images in our training and validation data

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt

# Data Loading

To build our image classifier, we begin by downloading the dataset. The dataset we are using is a filtered version of <a href="https://www.kaggle.com/c/dogs-vs-cats/data" target="_blank">Dogs vs. Cats</a> dataset from Kaggle (ultimately, this dataset is provided by Microsoft Research).

In previous Colabs, we've used <a href="https://www.tensorflow.org/datasets" target="_blank">TensorFlow Datasets</a>, which is a very easy and convenient way to use datasets. In this Colab however, we will make use of the class `tf.keras.preprocessing.image.ImageDataGenerator` which will read data from disk. We therefore need to directly download *Dogs vs. Cats* from a URL and unzip it to the Colab filesystem.

In [ ]:
_URL = 'https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip'

zip_dir = tf.keras.utils.get_file('cats_and_dogs_filterted.zip', origin=_URL, extract=True)

The dataset we have downloaded has following directory structure.

<pre style="font-size: 10.0pt; font-family: Arial; line-height: 2; letter-spacing: 1.0pt;" >
<b>cats_and_dogs_filtered</b>
|__ <b>train</b>
    |______ <b>cats</b>: [cat.0.jpg, cat.1.jpg, cat.2.jpg ....]
    |______ <b>dogs</b>: [dog.0.jpg, dog.1.jpg, dog.2.jpg ...]
|__ <b>validation</b>
    |______ <b>cats</b>: [cat.2000.jpg, cat.2001.jpg, cat.2002.jpg ....]
    |______ <b>dogs</b>: [dog.2000.jpg, dog.2001.jpg, dog.2002.jpg ...]
</pre>

We'll now assign variables with the proper file path for the training and validation sets.

In [ ]:
base_dir = os.path.join(os.path.dirname(zip_dir), 'cats_and_dogs_filterted_extracted/cats_and_dogs_filtered')
train_dir = os.path.join(base_dir, 'train')
validation_dir = os.path.join(base_dir, 'validation')

In [ ]:
train_cats_dir = os.path.join(train_dir, 'cats')  # directory with our training cat pictures
train_dogs_dir = os.path.join(train_dir, 'dogs')  # directory with our training dog pictures
validation_cats_dir = os.path.join(validation_dir, 'cats')  # directory with our validation cat pictures
validation_dogs_dir = os.path.join(validation_dir, 'dogs')  # directory with our validation dog pictures

### Understanding our data

Let's look at how many cats and dogs images we have in our training and validation directory

In [ ]:
num_cats_tr = len(os.listdir(train_cats_dir))
num_dogs_tr = len(os.listdir(train_dogs_dir))

num_cats_val = len(os.listdir(validation_cats_dir))
num_dogs_val = len(os.listdir(validation_dogs_dir))

total_train = num_cats_tr + num_dogs_tr
total_val = num_cats_val + num_dogs_val

In [ ]:
print('total training cat images:', num_cats_tr)
print('total training dog images:', num_dogs_tr)

print('total validation cat images:', num_cats_val)
print('total validation dog images:', num_dogs_val)
print("--")
print("Total training images:", total_train)
print("Total validation images:", total_val)

# Setting Model Parameters

For convenience, let us set up variables that will be used later while pre-processing our dataset and training our network.

In [ ]:
BATCH_SIZE = 100
IMG_SHAPE  = 150 # Our training data consists of images with width of 150 pixels and height of 150 pixels

After defining our generators for training and validation images, **flow_from_directory** method will load images from the disk and will apply rescaling and will resize them into required dimensions using single line of code.

# Data Augmentation

Overfitting often occurs when we have a small number of training examples. One way to fix this problem is to augment our dataset so that it has sufficient number and variety of training examples. Data augmentation takes the approach of generating more training data from existing training samples, by augmenting the samples through random transformations that yield believable-looking images. The goal is that at training time, your model will never see the exact same picture twice. This exposes the model to more aspects of the data, allowing it to generalize better.

In **tf.keras** we can implement this using the same **ImageDataGenerator** class we used before. We can simply pass different transformations we would want to our dataset as a form of arguments and it will take care of applying it to the dataset during our training process.

To start off, let's define a function that can display an image, so we can see the type of augmentation that has been performed. Then, we'll look at specific augmentations that we'll use during training.

In [ ]:
# This function will plot images in the form of a grid with 1 row and 5 columns where images are placed in each column.
def plotImages(images_arr):
    fig, axes = plt.subplots(1, 5, figsize=(20,20))
    axes = axes.flatten()
    for img, ax in zip(images_arr, axes):
        ax.imshow(img)
    plt.tight_layout()
    plt.show()

### Flipping the image horizontally

We can begin by randomly applying horizontal flip augmentation to our dataset and seeing how individual images will look after the transformation. This is achieved by passing `horizontal_flip=True` as an argument to the `ImageDataGenerator` class.

In [ ]:
image_gen = ImageDataGenerator(rescale=1./255, horizontal_flip=True)

train_data_gen = image_gen.flow_from_directory(batch_size=BATCH_SIZE,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_SHAPE,IMG_SHAPE))

To see the transformation in action, let's take one sample image from our training set and repeat it five times. The augmentation will be randomly applied (or not) to each repetition.

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

### Rotating the image

The rotation augmentation will randomly rotate the image up to a specified number of degrees. Here, we'll set it to 45.

In [ ]:
image_gen = ImageDataGenerator(rescale=1./255, rotation_range=45)

train_data_gen = image_gen.flow_from_directory(batch_size=BATCH_SIZE,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_SHAPE, IMG_SHAPE))

To see the transformation in action, let's once again take a sample image from our training set and repeat it. The augmentation will be randomly applied (or not) to each repetition.

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

### Applying Zoom

We can also apply Zoom augmentation to our dataset, zooming images up to 50% randomly.

In [ ]:
image_gen = ImageDataGenerator(rescale=1./255, zoom_range=0.5)

train_data_gen = image_gen.flow_from_directory(batch_size=BATCH_SIZE,
                                               directory=train_dir,
                                               shuffle=True,
                                               target_size=(IMG_SHAPE, IMG_SHAPE))

One more time, take a sample image from our training set and repeat it. The augmentation will be randomly applied (or not) to each repetition.

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

### Putting it all together

We can apply all these augmentations, and even others, with just one line of code, by passing the augmentations as arguments with proper values.

Here, we have applied rescale, rotation of 45 degrees, width shift, height shift, horizontal flip, and zoom augmentation to our training images.

In [ ]:
image_gen_train = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

train_data_gen = image_gen_train.flow_from_directory(batch_size=BATCH_SIZE,
                                                     directory=train_dir,
                                                     shuffle=True,
                                                     target_size=(IMG_SHAPE,IMG_SHAPE),
                                                     class_mode='binary')

Let's visualize how a single image would look like five different times, when we pass these augmentations randomly to our dataset. 

In [ ]:
augmented_images = [train_data_gen[0][0][0] for i in range(5)]
plotImages(augmented_images)

### Creating Validation Data generator

Generally, we only apply data augmentation to our training examples, since the original images should be representative of what our model needs to manage. So, in this case we are only rescaling our validation images and converting them into batches using ImageDataGenerator.

In [ ]:
image_gen_val = ImageDataGenerator(rescale=1./255)

val_data_gen = image_gen_val.flow_from_directory(batch_size=BATCH_SIZE,
                                                 directory=validation_dir,
                                                 target_size=(IMG_SHAPE, IMG_SHAPE),
                                                 class_mode='binary')

# Model Creation

## Define the model

The model consists of four convolution blocks with a max pool layer in each of them.

Before the final Dense layers, we're also applying a Dropout probability of 0.5. It means that 50% of the values coming into the Dropout layer will be set to zero. This helps to prevent overfitting.

Then we have a fully connected layer with 512 units, with a `relu` activation function. The model will output class probabilities for two classes — dogs and cats — using `softmax`. 

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(2)
])

### Compiling the model

As usual, we will use the `adam` optimizer. Since we output a softmax categorization, we'll use `sparse_categorical_crossentropy` as the loss function. We would also like to look at training and validation accuracy on each epoch as we train our network, so we are passing in the metrics argument.

In [ ]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

### Model Summary

Let's look at all the layers of our network using **summary** method.

In [ ]:
model.summary()

### Train the model

It's time we train our network.

Since our batches are coming from a generator (`ImageDataGenerator`), we'll use `fit_generator` instead of `fit`.

In [ ]:
epochs=1
history = model.fit(
    train_data_gen,
    steps_per_epoch=int(np.ceil(total_train / float(BATCH_SIZE))),
    epochs=epochs,
    validation_data=val_data_gen,
    validation_steps=int(np.ceil(total_val / float(BATCH_SIZE))))

### Visualizing results of the training

We'll now visualize the results we get after training our network.

In [ ]:
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(epochs)

plt.figure(figsize=(8, 8))
plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.show()

##### Copyright 2018 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Image Classification using tf.keras

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l05c04_exercise_flowers_with_data_augmentation_solution.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/examples/blob/master/courses/udacity_intro_to_tensorflow_for_deep_learning/l05c04_exercise_flowers_with_data_augmentation_solution.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
</table>

In this Colab you will classify images of flowers. You will build an image classifier using `tf.keras.Sequential` model and load data using `tf.keras.preprocessing.image.ImageDataGenerator`.


# Importing Packages

Let's start by importing required packages. **os** package is used to read files and directory structure, **numpy** is used to convert python list to numpy array and to perform required matrix operations and **matplotlib.pyplot** is used to plot the graph and display images in our training and validation data.

In [ ]:
import os
import numpy as np
import glob
import shutil
import matplotlib.pyplot as plt

### TODO: Import TensorFlow and Keras Layers

In the cell below, import Tensorflow and the Keras layers and models you will use to build your CNN. Also, import the `ImageDataGenerator` from Keras so that you can perform image augmentation.

In [ ]:
import tensorflow as tf

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Data Loading

In order to build our image classifier, we can begin by downloading the flowers dataset. We first need to download the archive version of the dataset and after the download we are storing it to "/tmp/" directory.

After downloading the dataset, we need to extract its contents.

In [ ]:
_URL = "https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz"

zip_file = tf.keras.utils.get_file(origin=_URL,
                                   fname="flower_photos",
                                   extract=True)

base_dir = os.path.join(os.path.dirname(zip_file), 'flower_photos/flower_photos')

The dataset we downloaded contains images of 5 types of flowers:

1. Rose
2. Daisy
3. Dandelion
4. Sunflowers
5. Tulips

So, let's create the labels for these 5 classes: 

In [ ]:
classes = ['roses', 'daisy', 'dandelion', 'sunflowers', 'tulips']

Also, the dataset we have downloaded has following directory structure. n
<pre style="font-size: 10.0pt; font-family: Arial; line-height: 2; letter-spacing: 1.0pt;" >
<b>flower_photos</b>
|__ <b>daisy</b>
|__ <b>dandelion</b>
|__ <b>roses</b>
|__ <b>sunflowers</b>
|__ <b>tulips</b>
</pre>

As you can see there are no folders containing training and validation data. Therefore, we will have to create our own training and validation set. Let's write some code that will do this.


The code below creates a `train` and a `val` folder each containing 5 folders (one for each type of flower). It then moves the images from the original folders to these new folders such that 80% of the images go to the training set and 20% of the images go into the validation set. In the end our directory will have the following structure:


<pre style="font-size: 10.0pt; font-family: Arial; line-height: 2; letter-spacing: 1.0pt;" >
<b>flower_photos</b>
|__ <b>daisy</b>
|__ <b>dandelion</b>
|__ <b>roses</b>
|__ <b>sunflowers</b>
|__ <b>tulips</b>
|__ <b>train</b>
    |______ <b>daisy</b>: [1.jpg, 2.jpg, 3.jpg ....]
    |______ <b>dandelion</b>: [1.jpg, 2.jpg, 3.jpg ....]
    |______ <b>roses</b>: [1.jpg, 2.jpg, 3.jpg ....]
    |______ <b>sunflowers</b>: [1.jpg, 2.jpg, 3.jpg ....]
    |______ <b>tulips</b>: [1.jpg, 2.jpg, 3.jpg ....]
 |__ <b>val</b>
    |______ <b>daisy</b>: [507.jpg, 508.jpg, 509.jpg ....]
    |______ <b>dandelion</b>: [719.jpg, 720.jpg, 721.jpg ....]
    |______ <b>roses</b>: [514.jpg, 515.jpg, 516.jpg ....]
    |______ <b>sunflowers</b>: [560.jpg, 561.jpg, 562.jpg .....]
    |______ <b>tulips</b>: [640.jpg, 641.jpg, 642.jpg ....]
</pre>

Since we don't delete the original folders, they will still be in our `flower_photos` directory, but they will be empty. The code below also prints the total number of flower images we have for each type of flower. 

In [ ]:
for cl in classes:
  img_path = os.path.join(base_dir, cl)
  images = glob.glob(img_path + '/*.jpg')
  print("{}: {} Images".format(cl, len(images)))
  num_train = int(round(len(images)*0.8))
  train, val = images[:num_train], images[num_train:]

  for t in train:
    if not os.path.exists(os.path.join(base_dir, 'train', cl)):
      os.makedirs(os.path.join(base_dir, 'train', cl))
    shutil.move(t, os.path.join(base_dir, 'train', cl))

  for v in val:
    if not os.path.exists(os.path.join(base_dir, 'val', cl)):
      os.makedirs(os.path.join(base_dir, 'val', cl))
    shutil.move(v, os.path.join(base_dir, 'val', cl))

In [ ]:
round(len(images)*0.8)

For convenience, let us set up the path for the training and validation sets

In [ ]:
train_dir = os.path.join(base_dir, 'train')
val_dir = os.path.join(base_dir, 'val')